In [ ]:
from lmptools import Dump, DumpCallback, DumpSnapshot
from lmptools.exceptions import SkipSnapshot
from typing import List
import numpy as np

In [ ]:
dumpfile1 = "/storage/backups/seagate/polymer_melts/simulations/melts_non_thermosttated_runs/one_epsilon/phi0d2_r125_p1000/run1/dump.rigidChains.lammpstrj"
dumpfile2 = "/storage/backups/seagate/polymer_melts/simulations/melts_non_thermosttated_runs/one_epsilon/phi0d2_r125_p1000/run1/dump.flexChains.lammpstrj"

In [ ]:
class PrintProgress(DumpCallback):
    def __init__(self, selected_timestamps):
        self.timestamps = selected_timestamps

    def on_snapshot_parse_timestamp(self, timestamp: int, *args, **kwargs):
        if timestamp not in self.timestamps:
            raise SkipSnapshot(f"Snapshot skipped for {timestamp}")

    def on_snapshot_parse_end(self, snapshot: DumpSnapshot, *args, **kwargs):
        print(f"Snapshot {snapshot.timestamp} parsed")

### Randomly select 50 timestamps from the raw dump files

In [ ]:
tselect = np.random.choice(np.arange(9000000, 10002000, 2000), 50)

In [ ]:
drigid = Dump(dumpfile1, unwrap=True, callback=PrintProgress(tselect))
dpolymer = Dump(dumpfile2, unwrap=True, callback=PrintProgress(tselect))

In [ ]:
with open('dump.melt.lammpstrj', 'w') as f:
    for rigid_snap, polymer_snap in zip(drigid, dpolymer):
        df = (rigid_snap+polymer_snap)
        f.write(str(df)+"\n")
f.close()